In [2]:
from django.shortcuts import render
from django.http import HttpResponse, Http404, JsonResponse

# from django.views.decorators.csrf import requires_csrf_token
# from django.views.decorators.csrf import csrf_exempt
# from django.views.decorators.csrf import ensure_csrf_cookie
# Create your views here.

import base64
import pytesseract as tess

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from keras.layers import *
from keras.models import Model
from tensorflow.keras.optimizers import Adam
import random
from keras.callbacks import ModelCheckpoint
import math
from collections import OrderedDict
import traceback
from keras import backend as K
from keras.models import load_model

from django.core.files.base import ContentFile

def index(request):
	data = request.GET["name"]
	return HttpResponse("hello "+data)

def getImage(request):
	# print("in method")
	# print("in if:::---")

	image_data = request.POST['image']
	#print("data: " + image_data)

	# image_data = readCode()
	data = base64.b64decode(image_data)

	file_name = '/Users/apple/Desktop/vishal/p1/ocr/media/demo1.jpg'
	with open(file_name,'wb') as f:
		f.write(data)

	img = file_name
	text = tess.image_to_string(img)
	dictData={"text":text}

	#print(text)
	return JsonResponse(dictData)


def readCode():
	file_name = '/Users/apple/Desktop/base64.txt'
	with open(file_name,"r") as base64Code:
		strCode = base64Code.read()

	return strCode

def imgToText_Tess(request):
	img = getImage()
	text = tess.image_to_string(img)
	dictData={"text":text}

	#print(text)
	return JsonResponse(dictData)



# ------------------------------------------------------------------------------
# IMAGE STRAIGHTEN :
# ------------------------------------------------------------------------------

def deskew(img):
    thresh=img
    edges = cv2.Canny(thresh,50,200,apertureSize = 3)
    
    lines = cv2.HoughLines(edges,1,np.pi/1000, 55)
    try:
        d1 = OrderedDict()
        for i in range(len(lines)):
            for rho,theta in lines[i]:
                deg = np.rad2deg(theta)
#                print(deg)
                if deg in d1:
                    d1[deg] += 1
                else:
                    d1[deg] = 1
                    
        t1 = OrderedDict(sorted(d1.items(), key=lambda x:x[1] , reverse=False))
        print(list(t1.keys())[0],'Angle' ,thresh.shape)
        non_zero_pixels = cv2.findNonZero(thresh)
        center, wh, theta = cv2.minAreaRect(non_zero_pixels)
        angle=list(t1.keys())[0]
        if angle>160:
            angle=180-angle
        if angle<160 and angle>20:
            angle=12        
        root_mat = cv2.getRotationMatrix2D(center, angle, 1)
        rows, cols = img.shape
        rotated = cv2.warpAffine(img, root_mat, (cols, rows), flags=cv2.INTER_CUBIC)
        
    except:
        rotated=img
        pass
    return rotated

def unshear(img):

    gray = img
    thresh = img.copy()
    #print(thresh)
    # plt.imshow(thresh)
    # plt.show()
    trans = thresh.transpose()

    arr=[]
    for i in range(thresh.shape[1]):
        arr.insert(0,trans[i].sum())

    arr=[]
    for i in range(thresh.shape[0]):
        arr.insert(0,thresh[i].sum())
    
    y = thresh.shape[0]-1-np.nonzero(arr)[0][0]
    y_top = thresh.shape[0]-1-np.nonzero(arr)[0][-1]

    trans1 = thresh.transpose()
    sum1=[]
    for i in range(trans1.shape[0]):
        sum1.insert(i,trans1[i].sum())

    height = y - y_top
    max_value = 255*height
    prev_num = len([i for i in sum1 if i>=(0.6*max_value)])
    final_ang = 0

    # # print(arr)
    # # print(x,y)
    for ang in range(-25,25,3):
        thresh = gray.copy()
        #print(thresh[0].shape)
        #print(ang)
        print('Ang',ang)
        if ang>0:
            #print(ang)
            for i in range(y):
                temp = thresh[i]
                move = int((y-i)*(math.tan(math.radians(ang))))
                if move >= temp.size:
                    move = temp.size
                thresh[i][:temp.size-move]=temp[move:]
                thresh[i][temp.size-move:] = [0 for m in range(move)]
        else:
            #print(ang)
            for i in range(y):
                temp = thresh[i]
                move = int((y-i)*(math.tan(math.radians(-ang))))
                if move >= temp.size:
                    move = temp.size
                #print(temp[:-3])
                #print(temp[:temp.size-move].shape, thresh[i][move%temp.size:].shape)
                thresh[i][move:]=temp[:temp.size-move]
                thresh[i][:move]=[0 for m in range(move)]

#         plt.imshow(thresh)
#         plt.show()
        trans1 = thresh.transpose()
        sum1=[]
        for i in range(trans1.shape[0]):
            sum1.insert(i,trans1[i].sum())
        #print(sum1)
        num = len([i for i in sum1 if i>=(0.60*max_value)])
        #print(num, prev_num)
        if(num>=prev_num):
            prev_num=num
            final_ang = ang
        #plt.imshow(thresh)
        #plt.show()
    #print("final_ang:", final_ang)

    thresh= gray.copy()
    if final_ang>0:
        for i in range(y):
            temp = thresh[i]
            move = int((y-i)*(math.tan(math.radians(final_ang))))
            if move >= temp.size:
                move = temp.size
            thresh[i][:temp.size-move]=temp[move:]
            thresh[i][temp.size-move:] = [0 for m in range(move)]
    else:
        for i in range(y):
            temp = thresh[i]
            move = int((y-i)*(math.tan(math.radians(-final_ang))))
            #print(move)
            if move >= temp.size:
                move = temp.size
            thresh[i][move:]=temp[:temp.size-move]
            thresh[i][:move]=[0 for m in range(move)]

#    plt.imshow(thresh)
#    plt.show()
    return thresh


# In[Main]:

def pad_with(vector, pad_width, iaxis, kwargs):
     pad_value = kwargs.get('padder', 40)
     vector[:pad_width[0]] = pad_value
     vector[-pad_width[1]:] = pad_value
     return vector

def image_straighten():
    img = cv2.imread('/Users/apple/Desktop/vishal/p1/ocr/media/f.png',0)
   
    thresh = cv2.threshold(img,127,255,1)[1]
    thresh=np.pad(thresh, 100, pad_with, padder=0)

    # plt.imshow(thresh)
    # plt.show()
    rotated = deskew(thresh)
    sheared_img = unshear(thresh)
    
    ret, thresh = cv2.threshold(sheared_img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    #plt.imshow(thresh)
    #plt.show()
    cv2.imwrite('/Users/apple/Desktop/vishal/p1/ocr/media/data/f.png', thresh)
    #return HttpResponse('image straightened!')

# ------------------------------------------------------------------------------
# SEGMENTATION :
# ------------------------------------------------------------------------------

def findCapPoints(img):
    cpoints=[]
    dpoints=[]
    for i in range(img.shape[1]):
        col = img[:,i:i+1]
        k = col.shape[0]
        while k > 0:
            if col[k-1]==255:
                dpoints.append((i,k))
                break
            k-=1
        
        for j in range(col.shape[0]):
            if col[j]==255:
                cpoints.append((i,j))
                break
    return cpoints,dpoints


def wordSegment(textLines):
    wordImgList=[]
    counter=0
    cl=0
    for txtLine in textLines:
        gray = cv2.cvtColor(txtLine, cv2.COLOR_BGR2GRAY)
        th, threshed = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
        final_thr = cv2.dilate(threshed,None,iterations = 20)

        # plt.imshow(final_thr)
        # plt.show()
        
        contours, hierarchy = cv2.findContours(final_thr,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        boundingBoxes = [cv2.boundingRect(c) for c in contours]
        (contours, boundingBoxes) = zip(*sorted(zip(contours, boundingBoxes), key=lambda b: b[1][0], reverse=False))
       
        for cnt in contours:
            area = cv2.contourArea(cnt)
 
#            print area
            if area > 10000:
                print ('Area= ',area)
                x,y,w,h = cv2.boundingRect(cnt)
                print (x,y,w,h)
                letterBgr = txtLine[0:txtLine.shape[1],x:x+w]
                wordImgList.append(letterBgr)
 
                cv2.imwrite("/Users/apple/Desktop/vishal/p1/ocr/media/result/words/" + str(counter) +".jpg",letterBgr)
                counter=counter+1
        cl=cl+1
       
    return wordImgList

def fitToSize(thresh1):
    
    mask = thresh1 > 0
    coords = np.argwhere(mask)

    x0, y0 = coords.min(axis=0)
    x1, y1 = coords.max(axis=0) + 1   # slices are exclusive at the top
    cropped = thresh1[x0:x1,y0:y1]
    return cropped


def lineSegment(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th, threshed = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
   
    upper=[]
    lower=[]
    flag=True
    for i in range(threshed.shape[0]):

        col = threshed[i:i+1,:]
        cnt=0
        if flag:
            cnt=np.count_nonzero(col == 255)
            if cnt >0:
                upper.append(i)
                flag=False
        else:
            cnt=np.count_nonzero(col == 255)
            if cnt <2:
                lower.append(i)
                flag=True
    textLines=[]
    if len(upper)!= len(lower):lower.append(threshed.shape[0])
#    print upper
#    print lower
    for i in range(len(upper)):
        timg=img[upper[i]:lower[i],0:]
        
        if timg.shape[0]>5:
#            plt.imshow(timg)
#            plt.show()
            timg=cv2.resize(timg,((timg.shape[1]*5,timg.shape[0]*8)))
            textLines.append(timg)

    return textLines


def baselines(letter2, upoints, dpoints, h, w,letterGray):
    colu = []
    for i in range(len(upoints)):
        colu.append(upoints[i][1])
    
    maxyu = max(colu)
    minyu = min(colu)
    avgu = (maxyu + minyu) // 2
    meanu = np.around(np.mean(colu)).astype(int)
    print('Upper:: Max, min, avg, mean:: ',maxyu, minyu, avgu, meanu)

    cold = []
    for i in range(len(dpoints)):
        cold.append(dpoints[i][1])
    
    maxyd = max(cold)
    minyd = min(cold)
    avgd = (maxyd + minyd) // 2
    meand = np.around(np.mean(cold)).astype(int)
    print('Lower:: Max, min, avg, mean:: ',maxyd, minyd, avgd, meand)

    cn = []
    count = 0

    for i in range(h):
        for j in range(w):
            if(letterGray[i,j] == 255):
                count+=1
        if(count != 0):
            cn.append(count)
            count = 0    
    maxindex = cn.index(max(cn))
    print('Max pixels at: ',maxindex)


    cv2.line(letter2,(0,meanu),(w,meanu),(255,0,0),2)
    lb = 0
    if(maxindex > meand):
        lb = maxindex
        cv2.line(letter2,(0,maxindex),(w,maxindex),(255,0,0),2)
    else:
        lb = meand
        cv2.line(letter2,(0,meand),(w,meand),(255,0,0),2)
        
    # plt.imshow(letter2)
    # plt.show()
    return meanu, lb


def histogram(letter2, upper_baseline, lower_baseline, w):
    ##------------Making Histograms (Default)------------------------######
    cropped = letter2[upper_baseline:lower_baseline,0:w]
    # plt.imshow(cropped)
    # plt.show()
    colcnt = np.sum(cropped==255, axis=0)
    x = list(range(len(colcnt)))
    # plt.plot(colcnt)
    # plt.fill_between(x, colcnt, 1, facecolor='blue', alpha=0.5)
    # plt.show()  
    return colcnt 

def visualize(letter2, upper_baseline, lower_baseline, min_pixel_threshold, min_separation_threshold, min_round_letter_threshold, colcnt, letterGray, h):
    seg = []
    seg1 = []
    seg2 = []
   ## Check if pixel count is less than min_pixel_threshold, add segmentation point
    for i in range(len(colcnt)):
      if(colcnt[i] < min_pixel_threshold):
          seg1.append(i)
          
    ## Check if 2 consequtive seg points are greater than min_separation_threshold in distance
    for i in range(len(seg1)-1):
        if(seg1[i+1]-seg1[i] > min_separation_threshold):
            seg2.append(seg1[i])

    arr= []

    for i in (seg2):
        arr1 = []
        j = upper_baseline
        while(j <= lower_baseline):
            if(letterGray[j,i] == 255):
                arr1.append(1)
            else:
                arr1.append(0)
            j+=1
        arr.append(arr1)
    print('At arr Seg here: ', seg2)
    
    ones = []
    for i in (arr):
        ones1 = []
        for j in range(len(i)):
            if (i[j] == 1):
                ones1.append([j])
        ones.append(ones1)
    
    diffarr = []
    for i in (ones):
        diff = i[len(i)-1][0] - i[0][0]
        diffarr.append(diff)
    print('Difference array: ',diffarr)
    
    for i in range(len(seg2)):
        if(diffarr[i] < min_round_letter_threshold):
            seg.append(seg2[i])


    for i in range(len(seg)):
        letter3 = cv2.line(letter2,(seg[i],0),(seg[i],h),(255,0,0),2)
    
    print("Does it work::::")
    # plt.imshow(letter3)
    # plt.show()
    return seg  

def segmentCharacters(seg,lettergray, fn):
    s=0
    wordImgList = []

    for i in range(len(seg)):
        if i==0:
            s=seg[i]
            if s > 15:
                wordImg = lettergray[0:,0:s]
                cntx=np.count_nonzero(wordImg == 255) 
                print ('count',cntx)
                # plt.imshow(wordImg)
                # plt.show()
                fn=fn+1
            else:
                continue
        elif (i != (len(seg)-1)):
            if seg[i]-s > 15:
                wordImg = lettergray[0:,s:seg[i]]
                cntx=np.count_nonzero(wordImg == 255) 
                print ('count',cntx)
                # plt.imshow(wordImg)
                # plt.show()
                fn=fn+1
                s=seg[i]
            else:
                continue
        else:
            wordImg = lettergray[0:,seg[len(seg)-1]:]
            cntx=np.count_nonzero(wordImg == 255) 
            print ('count',cntx)
            # plt.imshow(wordImg)
            # plt.show()
            fn=fn+1
        wordImgList.append(wordImg)

    return wordImgList

def segmentImg():

	fn = 0
	image_straighten() 
	path='/Users/apple/Desktop/vishal/p1/ocr/media/result/'

	#Taking any image from the sample images
	#In case of slanted image, straighten it using image-straighten.py, then use it
	img = cv2.imread('/Users/apple/Desktop/vishal/p1/ocr/media/f.png')

	try:
	    textLines=lineSegment(img)
	    print ('No. of Lines',len(textLines))
	    imgList=wordSegment(textLines)
	    print ('No. of Words',len(imgList))
	    counter = 0
	    for letterGray in imgList:
	        print ('LetterGray shape: ',letterGray.shape)
	        gray = cv2.cvtColor(letterGray, cv2.COLOR_BGR2GRAY)
	        th, letterGray = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
	        letterGray = fitToSize(letterGray)
	        letter2 = letterGray.copy()
	        letterGray = cv2.dilate(letterGray,None,iterations = 4)

	        h = letterGray.shape[0]
	        w = letterGray.shape[1]
	        
	        upoints, dpoints=findCapPoints(letterGray)        
	        meanu, lb = baselines(letter2, upoints, dpoints, h, w, letterGray)
	     
	        upper_baseline = meanu
	        lower_baseline = lb
	        
	        colcnt = histogram(letter2, upper_baseline, lower_baseline, w)

	        min_pixel_threshold = 25
	        min_separation_threshold = 35
	        min_round_letter_threshold = 190
	        
	        seg = visualize(letter2, upper_baseline, lower_baseline, min_pixel_threshold, min_separation_threshold, min_round_letter_threshold, colcnt, letterGray, h)
	        wordImgList = segmentCharacters(seg,letterGray, fn)
	        for i in wordImgList:
	            cv2.imwrite("/Users/apple/Desktop/vishal/p1/ocr/media/result/characters/" + str(counter) +".jpeg",i)
	            counter=counter+1
	             
	        
	    print('Original Image')  
	    #return HttpResponse('segment image success!')      
	    # plt.imshow(img)
	    # plt.show()

	except Exception as e:
	    print ('Error Message ',e)
	    cv2.destroyAllWindows()
	    traceback.print_exc()
	    #return HttpResponse('segment image error!') 
	    pass

	traceback.print_exc()

# ------------------------------------------------------------------------------
# CENTER ALIGN :
# ------------------------------------------------------------------------------

def add_padding(img, pad_l, pad_t, pad_r, pad_b):
    
    height, width = img.shape
    #Adding padding to the left side.
    pad_left = np.zeros((height, pad_l), dtype = np.int)
    img = np.concatenate((pad_left, img), axis = 1)

    #Adding padding to the top.
    pad_up = np.zeros((pad_t, pad_l + width))
    img = np.concatenate((pad_up, img), axis = 0)

    #Adding padding to the right.
    pad_right = np.zeros((height + pad_t, pad_r))
    img = np.concatenate((img, pad_right), axis = 1)

    #Adding padding to the bottom
    pad_bottom = np.zeros((pad_b, pad_l + width + pad_r))
    img = np.concatenate((img, pad_bottom), axis = 0)

    return img


def center_align():
	images = []
	segmentImg()
	folder='/Users/apple/Desktop/vishal/p1/ocr/media/result/characters/'
	for filename in os.listdir(folder):
	    img = cv2.imread(os.path.join(folder,filename),0)
	    th, a = cv2.threshold(img, 127, 255,cv2.THRESH_OTSU)
	    if a is not None:
	        a=cv2.resize(a,(100,80))
	        # create blank image - y, x
	        col_sum = np.where(np.sum(a, axis = 0)>0)
	        row_sum = np.where(np.sum(a, axis = 1)>0)
	        y1, y2 = row_sum[0][0], row_sum[0][-1]
	        x1, x2 = col_sum[0][0], col_sum[0][-1]
	        
	        cropped_image = a[y1:y2, x1:x2]        
	        cropped_image=cv2.resize(cropped_image,(20,20))
	        padded_image = add_padding(cropped_image, 4, 4, 4, 4)
	        cv2.imwrite('/Users/apple/Desktop/vishal/p1/ocr/media/result/resized_images/'+filename,padded_image)

	return HttpResponse("Images resized and saved into designated folder")


def recognition(request):
	center_align()
	K.set_image_data_format('channels_first')

	#Load model

	model=load_model('/Users/apple/Desktop/vishal/p1/ocr/model/Mnist1L_5Conv.h5')
	print(model.summary())

	import string
	letter_count = dict(zip(string.ascii_lowercase, range(1,27)))
	print('Letter_count: ',letter_count.items())


	x=[]
	res=[]
	fname=[]
	folder='/Users/apple/Desktop/vishal/p1/ocr/media/result/resized_images/'
	dirFiles=os.listdir(folder)
	dirFiles = sorted(dirFiles,key=lambda x: int(os.path.splitext(x)[0]))
	for filename in dirFiles:
	    imt = cv2.imread(os.path.join(folder,filename))
	    imt = cv2.blur(imt,(6,6))
	    gray = cv2.cvtColor(imt,cv2.COLOR_BGR2GRAY)
	    ret, imt = cv2.threshold(gray,0,255,cv2.THRESH_OTSU)
	    if imt is not None:
	        imt = imt.reshape((-1, 28, 28))
	#        plt.imshow(imt)
	#        plt.show()
	        imt=imt/255
	        x.append(imt)
	        fname.append(filename)
	       
	x=np.array(x);    
	predictions = model.predict(x)
	classes = np.argmax(predictions, axis=1)    

	for i in range(len(classes)):
	    imt = cv2.imread(os.path.join(folder,dirFiles[i]))
	    # plt.imshow(imt)
	    # plt.show()
	    print([k for k,v in letter_count.items() if v == classes[i]])





